In [1]:
# Change the dataset in global_vars.py to KORA.
from global_vars import *
from func import *

import pickle as p
import glob 
import json
import subprocess
import getpass
import os
from nibabel.orientations import axcodes2ornt, ornt_transform, inv_ornt_aff, flip_axis
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

In [2]:
# from nibabel.orientations import axcodes2ornt, ornt_transform, inv_ornt_aff, flip_axis
# vol = nb.load(f'{processed_dir}/volume/KORA2453666.nii.gz')
# label = nb.load(f'{processed_dir}/label/KORA2453666.nii.gz')

# label_d = flip_axis(label.get_fdata(), axis=0)
# label_d = flip_axis(label_d, axis=2)
# label = nb.Nifti1Image(label_d, label.affine, label.header)

# print('Viewing Stitched Images.....')
# volume_3_view_viewer(get_volume_data(vol))
# volume_3_view_viewer(get_volume_data(label))

# # save_volume(volume, f'{processed_dir}/volume/KORA2453666')
# save_volume(label, f'{processed_dir}/label/KORA2453666')

In [3]:
# Culprit 13 temp/KORA/processed_resized/label/KORA2460723.nii.gz
# img = nb.load('temp/KORA/processed_resized/label/KORA2460723.nii.gz')
# img_data= img.get_fdata()
# img_data[0:188, 70:80, 228:232] = 0
# empty_header = nibabel.Nifti1Header()
# all_label_img = nb.Nifti1Image(img_data, img.affine, img.header)
# save_volume(all_label_img, f'temp/KORA/processed_resized/label/KORA2460723_altered')

def crop_roi(paths):
    affine = None
    data = np.zeros((256,256,400))
    for idx, path in enumerate(paths):
#         if '1004985_20201_2_0' in path:
#             print('skipping :', idx)
#             continue
        print(path)
        img = nb.load(path)
        if affine is None:
            affine = img.affine
#         img = resample_from_to(img, [(300, 300, 300), img.affine])
        img_data= img.get_fdata()
        print(idx, path)
#         img_data[img_data!=0] = idx
#         img_data = np.round(img_data)
        print(np.unique(img_data))
        x,y,z = img.shape
        print(x,y,z)
        data[:x, :y, :z] += img_data
        
    empty_header = nibabel.Nifti1Header()
    all_label_img = nb.Nifti1Image(data, affine, empty_header)
    return all_label_img

def crop(paths, shape):
    s1, e1, s2, e2, s3, e3 = shape
    for path in paths:
        print(path)
        img = nb.load(path)
        print(img.shape)
#         img = resample_from_to(img, [(300, 300, 300), img.affine])
        img_data= img.get_fdata()
        data = img_data[s1:e1, s2:e2, s3:e3]
        img = nb.Nifti1Image(data, img.affine, img.header)
        print(img.shape)
        save_path = '/'.join(path.split('/')[:-1])
        vol_id = path.split('/')[-1].split('.')[0]
        save_volume(img, f'{save_path}_cropped/{vol_id}')

def remove_black(labelmap):
    clean_labels = []
    start, end = None, None
    start_2, end_2 = None, None
    for i, frame in enumerate(labelmap):
        unique, counts = np.unique(frame, return_counts=True)
#         print(counts)
#         if counts[3] < 2:
#             continue
        if counts[0] / sum(counts) < .99:
            clean_labels.append(frame)
            if start is None:
                start = i
            elif start_2 is None:
                start_2 = i
            if end is not None:
                end_2 = end
            end = None
        if end is None:
            end = i
    if end is None:
        end = labelmap.shape[0]
            
    return np.array(clean_labels), start, end, start_2, end_2




In [4]:
# processed_dir = 'temp/KORA/processed'
# kora_labels = glob.glob(f'{processed_dir}/label/**')
# kora_volumes = glob.glob(f'{processed_dir}/volume/**')
processed_dir = 'temp/KORA/n4_corrected_2'
# nako_labels = glob.glob(f'{processed_dir}/label/**')
nako_volumes = glob.glob(f'{processed_dir}/vol/**/OPP_**')
# processed_dir = 'temp/UKB/processed'
# ukb_labels = glob.glob(f'{processed_dir}/label/**')
# ukb_volumes = glob.glob(f'{processed_dir}/volume/**')

# all_labels = kora_labels + nako_labels + ukb_labels
# all_volumes = kora_volumes + nako_volumes + ukb_volumes

# print(all_labels, all_volumes)
print(nako_volumes)

['temp/KORA/n4_corrected_2/vol/KORA2459692/OPP_ras_stitched.nii.gz', 'temp/KORA/n4_corrected_2/vol/KORA2456230/OPP_ras_stitched.nii.gz', 'temp/KORA/n4_corrected_2/vol/KORA2456507/OPP_ras_stitched.nii.gz', 'temp/KORA/n4_corrected_2/vol/KORA2459431/OPP_ras_stitched.nii.gz']


In [5]:
def resize(img, shape=(256, 256, 400), is_label=False):
    if is_label:
        order = 0
    else:
        order = 3
    mode='constant'
    print(img.shape)
    img = resample_from_to(img, [shape, img.affine], order=order, mode=mode, cval=0)
    return img

In [6]:
for vol in nako_volumes:
#     vol_id = vol.split('/')[-1].split('.')[0]
    vol_id = vol.split('/')[-2]
    save_path = '/'.join(vol.split('/')[:-2])
    print(vol_id, save_path)
    vol_img = nb.load(vol)
    resized_img = resize(vol_img)
    save_volume(resized_img, f'{save_path}_resized/volume/{vol_id}')

KORA2459692 temp/KORA/n4_corrected_2/vol
(246, 123, 239)
saving directory: temp/KORA/n4_corrected_2/vol_resized/volume
KORA2456230 temp/KORA/n4_corrected_2/vol
(246, 137, 239)
saving directory: temp/KORA/n4_corrected_2/vol_resized/volume
KORA2456507 temp/KORA/n4_corrected_2/vol
(246, 191, 239)
saving directory: temp/KORA/n4_corrected_2/vol_resized/volume
KORA2459431 temp/KORA/n4_corrected_2/vol
(246, 137, 239)
saving directory: temp/KORA/n4_corrected_2/vol_resized/volume


In [ ]:
# for vol in ukb_labels:
#     vol_id = vol.split('/')[-1].split('.')[0]
#     save_path = '/'.join(vol.split('/')[:-2])
#     print(vol_id, save_path)
#     vol_img = nb.load(vol)
#     resized_img = resize(vol_img, is_label=True)
#     save_volume(resized_img, f'{save_path}_resized/label/{vol_id}')

In [7]:
# processed_resized_dir = 'temp/KORA/processed_resized'
# kora_labels = glob.glob(f'{processed_resized_dir}/label/**')
# kora_volumes = glob.glob(f'{processed_resized_dir}/volume/**')
processed_resized_dir = 'temp/KORA/n4_corrected_2/vol_resized'
# nako_labels = glob.glob(f'{processed_resized_dir}/label/**')
nako_volumes = glob.glob(f'{processed_resized_dir}/volume/**')
# processed_resized_dir = 'temp/UKB/processed_resized'
# ukb_labels = glob.glob(f'{processed_resized_dir}/label/**')
# ukb_volumes = glob.glob(f'{processed_resized_dir}/volume/**')

In [8]:
nako_volumes

['temp/KORA/n4_corrected_2/vol_resized/volume/KORA2459431.nii.gz',
 'temp/KORA/n4_corrected_2/vol_resized/volume/KORA2459692.nii.gz',
 'temp/KORA/n4_corrected_2/vol_resized/volume/KORA2456507.nii.gz',
 'temp/KORA/n4_corrected_2/vol_resized/volume/KORA2456230.nii.gz']

In [ ]:
# kls = crop_roi(kora_labels)
# volume_3_view_viewer(get_volume_data(kls))
# save_volume(kls, f'all_labels_merged/all_kora_labels')

# kls = crop_roi(nako_labels)
# volume_3_view_viewer(get_volume_data(kls))
# save_volume(kls, f'all_labels_merged/all_nako_labels')

kls = crop_roi(ukb_labels)
# resample_to_output()
volume_3_view_viewer(get_volume_data(kls))
save_volume(kls, f'all_labels_merged/all_ukb_labels')

In [ ]:
# img = nb.load(f'all_labels_merged/all_kora_labels.nii.gz')
# img_data = img.get_fdata()
# img_d1, s1,e1, s12, e12 = remove_black(img_data)
# img_data1 = np.moveaxis(img_data, 1, 0)
# img_d2, s2,e2, s22, e22 = remove_black(img_data1)
# img_data2 = np.moveaxis(img_data, 2, 0)
# img_d3,s3,e3, s32, e32 = remove_black(img_data2)
# # img_data = np.moveaxis(img_data, axis=0)

# print('kora', img_d1.shape, img_d2.shape, img_d3.shape, s1,e1,s2,e2,s3,e3,'---', s12,e12,s22,e22,s32,e32)

# img = nb.load(f'all_labels_merged/all_nako_labels.nii.gz')
# img_data = img.get_fdata()
# img_d1,s1,e1, s12, e12 = remove_black(img_data)
# img_data1 = np.moveaxis(img_data, 1, 0)
# img_d2,s2,e2, s22, e22 = remove_black(img_data1)
# img_data2 = np.moveaxis(img_data, 2, 0)
# img_d3,s3,e3, s32, e32 = remove_black(img_data2)
# # img_data = np.moveaxis(img_data, axis=0)

# print('nako', img_d1.shape, img_d2.shape, img_d3.shape, s1,e1,s2,e2,s3,e3,'---', s12,e12,s22,e22,s32,e32)

img = nb.load(f'all_labels_merged/all_ukb_labels.nii.gz')
img_data = img.get_fdata()
img_d1,s1,e1, s12, e12 = remove_black(img_data)
img_data1 = np.moveaxis(img_data, 1, 0)
img_d2,s2,e2, s22, e22 = remove_black(img_data1)
img_data2 = np.moveaxis(img_data, 2, 0)
img_d3,s3,e3, s32, e32 = remove_black(img_data2)
# img_data = np.moveaxis(img_data, axis=0)

print('ukb', img_d1.shape, img_d2.shape, img_d3.shape, s1,e1, s2,e2, s3,e3,'---', s12,e12,s22,e22,s32,e32)

In [ ]:
# KORA = 158,100,91   ---  49, 206, 23, 122, 84, 176  --> 40, 210, 5, 145, 80, 180 == 170, 140, 100  == 176, 144, 112/96  (Multiples of 16)
        "40, 215, 5, 175, 70, 180" ==                                                   "175, 170, 110" ==  '192 176 112'
# NAKO = 158,104,76    --- 44, 201, 40, 143, 169, 244  --> 41, 201, 23, 143, 169, 244 == 160, 120, 75 == 160, 128, 84 =AFTER FAT TALL MATCH= 31, 201, 23, 143, 160, 265 == 
#                                                                                      "170, 120, 105"
# UKB= 163, 117, 97    --- 47, 209, 42, 158, 80, 176  --> 35, 215, 25, 170, 215, 315 == 180,145, 100 == 176/192, 144/160, 112 =AFTER FAT TALL[Missing] MATCH= 
#                                                       35, 215, 25, 170, 210, 315 == "180, 145, 105" (Evaluated on 6 merged UKB scans. BH+NOBH)
 
# KORA sample vol size = 250, 188, 370
# NAKO sample vol size = 226,184,316
# UKB sample vol size = 246, 137, 239
# sample target shape = 256, 256,400

nako_volumes

In [22]:
crop(nako_volumes, [40, 215, 5, 175, 70, 180])

temp/KORA/n4_corrected_2/vol_resized/volume/KORA2459431.nii.gz
(256, 256, 400)
(175, 170, 110)
saving directory: temp/KORA/n4_corrected_2/vol_resized/volume_cropped
temp/KORA/n4_corrected_2/vol_resized/volume/KORA2459692.nii.gz
(256, 256, 400)
(175, 170, 110)
saving directory: temp/KORA/n4_corrected_2/vol_resized/volume_cropped
temp/KORA/n4_corrected_2/vol_resized/volume/KORA2456507.nii.gz
(256, 256, 400)
(175, 170, 110)
saving directory: temp/KORA/n4_corrected_2/vol_resized/volume_cropped
temp/KORA/n4_corrected_2/vol_resized/volume/KORA2456230.nii.gz
(256, 256, 400)
(175, 170, 110)
saving directory: temp/KORA/n4_corrected_2/vol_resized/volume_cropped


In [ ]:
crop(nako_labels, [41, 201, 23, 143, 169, 244])